In [1]:
import matplotlib.pyplot as plt
import mlxtend
import numpy as np
import pandas as pd
import seaborn as sns

from mlxtend.preprocessing import TransactionEncoder
from sklearn.preprocessing import LabelEncoder


In [2]:
pd.set_option('display.max_row', 111)
pd.set_option('display.max_column', 111)
plt.style.use('dark_background')

data = pd.read_csv("~/Documents/DATASETS/students/KaDo.csv")
df = data.copy()[:5000]

In [3]:
# Delete unused columns, drop NAs, create additionnal columns, and only keep TICKET_IDS with multiple product which are not the same.
df.drop(columns=['MOIS_VENTE', 'PRIX_NET'], axis=1, inplace=True)
df.dropna(axis='columns')
# df['NB OF RELATIONS'] = 0
# df['RELATED CLI_IDs'] = None
encoder = LabelEncoder()
df['ALL_LIBELLES'] = None
df['CODE_LIBELLE'] = encoder.fit_transform(df['LIBELLE'])
# Keep only ticket ids with multiple product
df = df.drop_duplicates()
single_tickets = df.duplicated(subset='TICKET_ID', keep=False)
df = df[single_tickets]
# NB: Pour le ticket id 36417517 on peut remarquer qu'en produits achetés ensemble il y a une ressemblance dans le libelle où l'on retrouve LILAS.
# UPDATE : Même chose pour le ticket id 32975823....
# Peut-être intéréssant de concaténer la maille et le libelle et observer les ressemblances ?
df.head(30)

,TICKET_ID,FAMILLE,UNIVERS,MAILLE,LIBELLE,CLI_ID,ALL_LIBELLES,CODE_LIBELLE
0,35592159,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,None,367
2,35592159,SOINS DU VISAGE,VIS_CJOUR Jeunes Specifique,VIS_JEUNE_ET_LEVRE,CR JR PARF BIO.SPE AC.SENT.50ML,1490281,None,120
3,35592159,SOINS DU VISAGE,VIS_DEMAQ AAAR,VIS_AAAR_DEMAQLOTION,EAU MICELLAIRE 3 THES FL200ML,1490281,None,201
4,35592159,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 TIARE FL 200ML,1490281,None,369
5,35509899,PARFUMAGE,PARF_EAUX PARFUMS,PARF_PARFUM,EDT UN MATIN AU JARDIN 100ML MUGUET,13290776,None,232
6,35509899,SOINS DU CORPS,CORPS_LAIT HUILE PLAISIRNATURE,CORPS_HYDR_LAIT_HUILE,LAIT VELOUTE COCO PN2 400ML,13290776,None,473
7,36417517,HYGIENE,HYG_DOUCHE PARFUMS,HYG_PARFUMEE,GD LILAS FP FL200ML,13290776,None,370
10,36417517,SOINS DU CORPS,CORPS_LAIT HUILE PARFUMS,CORPS_HYDR_LAIT_HUILE,LAIT LILAS FP FL200ML,13290776,None,462
12,36417517,PARFUMAGE,PARF_EAUX PARFUMS,PARF_PARFUM,EDT UN MATIN AU JARDIN 100ML LILAS,13290776,None,258
15,33064616,SOINS DU VISAGE,VIS_CJOUR Jeunes Specifique,VIS_JEUNE_ET_LEVRE,CR JOUR PX/MIX HYDRA/VEG P50ML,20200041,None,116


In [4]:
# Keep in case
all_ticket_ids = df['TICKET_ID'].tolist()
all_libelles_list = []
for ticket_id in all_ticket_ids:
    all_libelles = df[df['TICKET_ID'] == ticket_id]['LIBELLE'].tolist()
    all_libelles_list.append(all_libelles)
df['ALL_LIBELLE'] = all_libelles_list

In [5]:
#Separate two dataframes, one for calculating the association rules and the other for looking up the libelles(description) to display on our recommendations
orders = df[['TICKET_ID', 'CODE_LIBELLE']]
products = df[['CODE_LIBELLE', 'LIBELLE']]

# Drop duplicated products
# products = products[~products.duplicated()]

# Set the index to CODE_LIBELLE
products = products.set_index('CODE_LIBELLE')# Convert to Series for eve easier lookups
products = products['LIBELLE']